In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# 1. 데이터 입출력 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

training_data = datasets.ImageFolder(
    root="data/train",
    transform=transform
)

test_data = datasets.ImageFolder(
    root="data/test", 
    transform=transform
)

batch_size = 32
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 2. 모델 정의
class TrashClassificationModel(nn.Module):
    def __init__(self, num_classes=6):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.AdaptiveAvgPool2d((7, 7))
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 7 * 7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = TrashClassificationModel()

# 3. 손실함수 정의
criterion = nn.CrossEntropyLoss()

# 4. 가중치 업데이트(학습)
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 20

def train_loop(dataloader, model, criterion, optimizer):
    model.train()
    size = len(dataloader.dataset)
    running_loss = 0.0
    
    for batch, (x, y) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * x.size(0)
        
        if (batch + 1) % 100 == 0:
            current = batch * len(x)
            print(f"[batch: {batch+1:4d}], Loss: {loss.item():>7f} ({current:>5d}/{size:>5d})")
    
    epoch_loss = running_loss / size
    return epoch_loss

def test_loop(dataloader, model, criterion):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0.0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            outputs = model(x)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            correct += (outputs.argmax(1) == y).sum().item()
    
    avg_loss = test_loss / num_batches
    accuracy = correct / size
    print(f"Test - Accuracy: {100*accuracy:>5.1f}%, Avg_loss:{avg_loss:>8f}")
    return avg_loss, accuracy

print()
for epoch in range(epochs):
    print(f"[Epoch] {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, criterion, optimizer)
    val_loss, val_acc = test_loop(test_dataloader, model, criterion)
print()
print("완료!!!!!!!")

# 5. 시각화
label_tags = {
    0: "Plastic", 1: "Paper", 2: "Glass", 3: "Metal", 4: "Organic", 5: "Cardboard"
}

rows, columns = 6, 6
fig = plt.figure(figsize=(15,15))
model.eval()

for i in range(1, rows * columns + 1):
    data_idx = np.random.randint(len(test_data))
    img_tensor, true_label = test_data[data_idx]
    
    with torch.no_grad():
        x = img_tensor.unsqueeze(0)
        output = model(x)
        pred_idx = output.argmax(1).item()
    
    pred_class = label_tags[pred_idx]
    true_class = label_tags[true_label]
    is_correct = (pred_idx == int(true_label))
    title = f"{pred_class}, correct!" if is_correct else f"{pred_class}, incorrect! answer: {true_class}"
    cmap = "Blues" if is_correct else "Reds"
    
    ax = fig.add_subplot(rows, columns, i)
    img_np = img_tensor.permute(1, 2, 0).numpy()
    img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img_np = np.clip(img_np, 0, 1)
    ax.imshow(img_np, cmap=cmap if len(img_np.shape) == 2 else None)
    ax.set_title(title, fontsize=10)
    ax.axis("off")

plt.tight_layout()
plt.show()

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'data/train'